In [1]:
%pip install libclang

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import glob
import json
import os
import pathlib
import re
from typing import Union, Iterator, List

import clang.cindex as ci
from clang.cindex import TokenKind as tk
from clang.cindex import CursorKind as ck

import ast_utils

jj = os.path.join

FS_ROOT = "/"
ROOT_DIR = jj(FS_ROOT, "/home/max/projects/autoware/")

def pext(t: Union[ci.Token, ci.Cursor]):
    e = t.extent
    return f"{e.start.line}:{e.start.column}-{e.end.line}:{e.end.column}"

def pt(tokens, join_str=''):
    if tokens is None:
        return "None"
    return join_str.join(map(lambda t: t.spelling, tokens))

if not os.path.exists(jj(ROOT_DIR, "build/compile_commands.json")):
    print("Run")
    print("   colcon build --symlink-install --cmake-args -DCMAKE_BUILD_TYPE=Release -DCMAKE_EXPORT_COMPILE_COMMANDS=ON")
    print("to generate the files necessary to run this script.")

In [3]:
_arg_pattern = re.compile(r"(?<=\s)(?:-D\s*\S+|-I\s*\S+|-isystem\s*\S+|--?std\s*\S+)(?=\s)")

def _extract_args(compile_command):
    segments = _arg_pattern.findall(compile_command)
    segments = [part.strip() for seg in segments for part in seg.split(" ")]
    return segments

with open(jj(ROOT_DIR, "build/compile_commands.json"), "r") as f:
    compile_commands = json.load(f)

compile_commands = {cmd['file']: cmd['command'] for cmd in compile_commands}
compile_commands = {k: _extract_args(v) for k, v in compile_commands.items()}

In [4]:
all_mappings = {}
idx = ci.Index.create()

for src_file, args in compile_commands.items():
    src_file: str
    tu = idx.parse(src_file, args=args)  # 0x200: KeepGoing (after fatal errors)
    h = ast_utils.TUHandler(tu)
    h.get_subscription_callback_handlers()

with open("cb_identifiers.json", "w") as f:
    json.dump(all_mappings, f)

['{ transform_listener_ = std :: make_shared < tier4_autoware_utils :: TransformListener > ( this ) ; // get parameter update_hz_ = this -> declare_parameter < double > ( "update_hz" , 10.0 ) ; covariance_ = this -> declare_parameter < double > ( "initial_covariance" , 0.05 ) ; velocity_min_threshold_ = this -> declare_parameter < double > ( "velocity_min_threshold" , 0.1 ) ; velocity_diff_threshold_ = this -> declare_parameter < double > ( "velocity_diff_threshold" , 0.556 ) ; pedal_diff_threshold_ = this -> declare_parameter < double > ( "pedal_diff_threshold" , 0.03 ) ; max_steer_threshold_ = this -> declare_parameter < double > ( "max_steer_threshold" , 0.2 ) ; max_pitch_threshold_ = this -> declare_parameter < double > ( "max_pitch_threshold" , 0.02 ) ; max_jerk_threshold_ = this -> declare_parameter < double > ( "max_jerk_threshold" , 0.7 ) ; pedal_velocity_thresh_ = this -> declare_parameter < double > ( "pedal_velocity_thresh" , 0.15 ) ; map_update_gain_ = this -> declare_param

KeyboardInterrupt: 